In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
url = "https://raw.githubusercontent.com/ilya-lysenko-An/Pet-EDA/main/london_merged.csv"
df = pd.read_csv(url)
print("Размер датафрейма:", df.shape)
print("Общие сведения о Дата Сете")
print(df.head().T)

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month
df[['timestamp', 'hour', 'dayofweek', 'month' ]].head().T

In [ ]:
monthly = df.groupby('month').agg({
    'cnt' : 'median',
    't1' : 'mean',
    'wind_speed' : 'mean',
    'hum' : 'mean'
})
print(monthly.round(2))

In [ ]:
hourly = df.groupby('hour').agg({
    'cnt' : 'median',
    't1' : 'mean',
    'wind_speed' : 'mean',
    'hum' : 'mean'
})
print(hourly.round(2))

In [ ]:
plt.figure(figsize=(12,5))
plt.bar(hourly.index, hourly['cnt'], color='skyblue')
plt.title("Медианное колличество поездок по часам")
plt.xlabel("Часы")
plt.ylabel("Колличество поездок")
plt.xticks(range(24))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,5))

ax1.bar(monthly.index, monthly['cnt'], color='skyblue', label='Поездки (медиана)')
ax1.set_xlabel("Месяц")
ax1.set_ylabel("Количество поездок", color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')
ax1.set_xticks(range(1,13))
ax1.set_title("Сравнение количества поездок и температуры по месяцам")
ax1.grid(axis='y', linestyle='--', alpha=0.5)

ax2 = ax1.twinx() 
ax2.plot(monthly.index, monthly['t1'], color='orange', marker='o', label='Температура (средняя)')
ax2.set_ylabel("Температура (°C)", color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

fig.tight_layout()
plt.show()


In [ ]:
mean_weekday = df[(df['is_weekend'] == 0) & (df['is_holiday'] == 0)]['cnt'].mean()
mean_weekend = df[(df['is_weekend'] == 1) & (df['is_holiday'] == 0)]['cnt'].mean()
mean_holiday = df[df['is_holiday'] == 1]['cnt'].mean()
print("Среднее колличество поездок")
print("В Будние дни:", mean_weekday.round(1))
print("В выходные дни:", mean_weekend.round(1))
print("В празднечные дни:", mean_holiday.round(1))
percent_weekend = ((mean_weekend - mean_weekday) / mean_weekday * 100).round(2)
percent_holiday = ((mean_holiday - mean_weekday) / mean_weekday * 100).round(2)
print("Насколько меньше в праздники и выходные берут велосипеды (в процентах)")
print("Выходные", percent_weekend)
print("Праздники", percent_holiday)

In [ ]:
labels = ['Будние', 'Выходные', 'Праздники']
values = [mean_weekday, mean_weekend, mean_holiday]

plt.figure(figsize=(8,5))
plt.bar(labels, values, color=['skyblue','orange','green'])
plt.ylabel("Среднее количество поездок")
plt.title("Сравнение поездок в будние, выходные и праздничные дни")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()


In [ ]:
plt.figure(figsize=(12,5))
plt.hist(df['cnt'], bins=50, color='skyblue', edgecolor='black')
plt.title("Распределение количества поездок")
plt.xlabel("Количество поездок")
plt.ylabel("Частота")
plt.show()

In [ ]:
numeric_cols = ['cnt', 't1', 't2', 'hum', 'wind_speed', 'weather_code', 'is_holiday', 'is_weekend', 'season']
corr = df[numeric_cols].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot = True, fmt='.2f', cmap = 'coolwarm', cbar = True)
plt.title("Кореляционная матрица признаков")
plt.show()

In [ ]:
weekday_df = df[(df['is_weekend'] == 0) & (df['is_holiday'] == 0)]
weekend_df = df[df['is_weekend'] == 1]

weekday_temp = (
    weekday_df
    .assign(t1_round = weekday_df['t1'].round())
    .groupby('t1_round')['cnt']
    .median()
)

weekend_temp = (
    weekend_df
    .assign(t1_round = weekend_df['t1'].round())
    .groupby('t1_round')['cnt']
    .median()
)